# Nonconvex quadratically constrained quadratic programs

**Adapted from**: [Hesse1973](@cite), [Floudas1999; Section 3.4](@cite), [Laurent2008; Example 6.22](@cite) and [Lasserre2009; Table 5.1](@cite)

We consider the nonconvex Quadratically Constrained Quadratic Programs (QCQP)
introduced in [H73].
Consider now the polynomial optimization problem [Laurent2008; Example 6.22](@cite) of
maximizing the convex quadratic function
(hence nonconvex since convex programs should either maximize concave functions
or minimize convex functions)
$25(x_1 - 2)^2 + (x_2 - 2)^2 + (x_3 - 1)^2 + (x_4 - 4)^2 + (x_5 - 1)^2 + (x_6 - 4)^2$
over the basic semialgebraic set defined by the nonconvex quadratic inequalities
$(x_3 - 3)^2 + x_4 \ge 4$,
$(x_5 - 3)^2 + x_6 \ge 4$,
and linear inequalities
$x_1 - 3x_2 \le 2$,
$-x_1 + x_2 \le 2$,
$2 \le x_1 + x_2 \le 6$,
$0 \le x_1, x_2$,
$1 \le x_3 \le 5$,
$0 \le x_4 \le 6$,
$1 \le x_5 \le 5$,
$0 \le x_6 \le 10$,
$x_2 \le 4x_1^4 - 32x_1^3 + 88x_1^2 - 96x_1 + 36$ and the box constraints
$0 \le x_1 \le 3$ and $0 \le x_2 \le 4$,

In [1]:
using DynamicPolynomials
@polyvar x[1:6]
centers = [2, 2, 1, 4, 1, 4]
weights = [25, 1, 1, 1, 1, 1]
p = -weights' * (x .- centers).^2
using SumOfSquares
K = @set x[1] >= 0 && x[2] >= 0 &&
    x[3] >= 1 && x[3] <= 5 &&
    x[4] >= 0 && x[4] <= 6 &&
    x[5] >= 1 && x[5] <= 5 &&
    x[6] >= 0 && x[6] <= 10 &&
    (x[3] - 3)^2 + x[4] >= 4 &&
    (x[5] - 3)^2 + x[6] >= 4 &&
    x[1] - 3x[2] <= 2 &&
    -x[1] + x[2] <= 2 &&
    x[1] + x[2] <= 6 &&
    x[1] + x[2] >= 2

Basic semialgebraic Set defined by no equality
16 inequalities
 x[1] ≥ 0
 x[2] ≥ 0
 -1 + x[3] ≥ 0
 5 - x[3] ≥ 0
 x[4] ≥ 0
 6 - x[4] ≥ 0
 -1 + x[5] ≥ 0
 5 - x[5] ≥ 0
 x[6] ≥ 0
 10 - x[6] ≥ 0
 5 + x[4] - 6*x[3] + x[3]^2 ≥ 0
 5 + x[6] - 6*x[5] + x[5]^2 ≥ 0
 2 + 3*x[2] - x[1] ≥ 0
 2 - x[2] + x[1] ≥ 0
 6 - x[2] - x[1] ≥ 0
 -2 + x[2] + x[1] ≥ 0


We will now see how to find the optimal solution using Sum of Squares Programming.
We first need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [2]:
import Clarabel
solver = Clarabel.Optimizer

Clarabel.MOIwrapper.Optimizer

A Sum-of-Squares certificate that $p \ge \alpha$ over the domain `S`, ensures that $\alpha$ is a lower bound to the polynomial optimization problem.
The following function searches for the largest lower bound and finds zero using the `d`th level of the hierarchy`.

In [3]:
function solve(d)
    model = SOSModel(solver)
    @variable(model, α)
    @objective(model, Max, α)
    @constraint(model, c, p >= α, domain = K, maxdegree = d)
    optimize!(model)
    println(solution_summary(model))
    return model
end

solve (generic function with 1 method)

The first level of the hierarchy cannot find any lower bound.

In [4]:
model2 = solve(2)
nothing # hide

-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 45
  constraints   = 72
  nnz(P)        = 0
  nnz(A)        = 109
  cones (total) = 3
    : Zero        = 1,  numel = 28
    : Nonnegative = 1,  numel = 16
    : PSDTriangle = 1,  numel = 28

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10



The second level of the hierarchy finds the lower bound of `-310`.

In [5]:
model3 = solve(4)
nothing # hide

-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 423
  constraints   = 506
  nnz(P)        = 0
  nnz(A)        = 1243
  cones (total) = 17
    : Zero        = 1,  numel = 84
    : Nonnegative = 1,  numel = 2
    : PSDTriangle = 15,  numel = (28,28,28,28,...,28)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
        

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*